In [11]:
# Import necessary libraries
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset

ModuleNotFoundError: No module named 'datasets'

In [6]:

# Load your dataset
# Replace 'dataset_name' and 'split' with your dataset and split name
dataset = load_dataset("dataset_name", "split")

# Define the tokenizer and model
model_name = "your_model_name"  # e.g., "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_total_limit=2,
    save_steps=500,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=100,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Define a function to preprocess the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define a function to compute metrics
def compute_metrics(p):
    return {"accuracy": (p["labels"] == p["predictions"]).mean()}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=None,
    compute_metrics=compute_metrics,
)


In [ ]:
# Train the model
trainer.train()

In [ ]:
# Evaluate the model
results = trainer.evaluate()

# Save the trained model
model.save_pretrained("./your_trained_model")

# Load the trained model
loaded_model = AutoModelForSequenceClassification.from_pretrained("./your_trained_model")

# Inference with the loaded model
# Replace 'your_input_text' with your actual input text
input_text = "your_input_text"
input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"]
output = loaded_model(**input_ids)

# Print the output
print(output)

# Fine-tune the model, if needed
# You can continue fine-tuning the loaded model if necessary
